### References: 
#### https://machinelearningmastery.com/how-to-develop-a-convolutional-neural-network-from-scratch-for-mnist-handwritten-digit-classification/

#### https://github.com/SadmanSakib93/Federated-Learning-Keras/blob/main/Fed%20Learning%20-%20FL.ipynb

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from keras.utils import np_utils
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten, BatchNormalization
from tensorflow.keras.layers import Convolution2D, Conv1D
from tensorflow.keras.layers import MaxPooling2D, MaxPooling1D
from keras import backend as K
from keras import backend
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import time
import os
import psutil
import csv
from itertools import repeat
from PIL import Image
from numpy import asarray

In [ ]:
tf.keras.datasets.mnist.load_data(path="mnist.npz")

In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [4]:
x_train = x_train.reshape((x_train.shape[0], 28, 28, 1))
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1))

In [5]:
x_train.shape

(60000, 28, 28, 1)

In [6]:
X_full = np.concatenate((x_train,x_test),axis=0)
X_full.shape

(70000, 28, 28, 1)

In [7]:
Y_full = np.concatenate((y_train,y_test),axis=0)
Y_full.shape

(70000,)

In [8]:
len(set(Y_full))

10

In [9]:
x_train.shape

(60000, 28, 28, 1)

In [10]:
algoName='CNN' #CNN, ANN, DNN

xTrain = x_train.astype('float32')
xTest = x_test.astype('float32')
xTrain = xTrain / 255.
xTest = xTest / 255.

# if(algoName=='CNN'):
#     xTrain = np.expand_dims(xTrain, axis=2)
#     xTest = np.expand_dims(xTest, axis=2)

outputClasses= len(set(Y_full))
#One hot encoding
yTrain = np.array(to_categorical(y_train))
yTest = np.array(to_categorical(y_test))
print("xTrain", xTrain.shape, "yTrain", yTrain.shape)
print("xTest", xTest.shape, "yTest", yTest.shape)

# FOR TEST SPLIT
xServer, xClients, yServer, yClients = train_test_split(xTrain, yTrain, test_size=0.80,random_state=523) 

xTrain (60000, 28, 28, 1) yTrain (60000, 10)
xTest (10000, 28, 28, 1) yTest (10000, 10)


In [11]:
def my_metrics(y_true, y_pred):
    accuracy=accuracy_score(y_true, y_pred)
    precision=precision_score(y_true, y_pred,average='weighted')
    recall=recall_score(y_true, y_pred,average='weighted')
    f1Score=f1_score(y_true, y_pred, average='weighted') 
    print("Accuracy  : {}".format(accuracy))
    print("Precision : {}".format(precision))
    print("Recall : {}".format(recall))
    print("f1Score : {}".format(f1Score))
    cm=confusion_matrix(y_true, y_pred)
    print(cm)
    return accuracy, precision, recall, f1Score

In [12]:
from keras.layers import Conv2D

In [13]:
from tensorflow.keras.optimizers import SGD

In [14]:
verbose, epochs, batch_size = 0, 20, 64
activationFun='relu'
optimizerName='Adam'
def createDeepModel():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(10, activation='softmax'))
    # compile model
    opt = SGD(learning_rate=0.01, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [15]:
def predictTestData(yPredict, yTest):
    #Converting predictions to label
    print("yPredict",len(yPredict))
    pred = list()
    for i in range(len(yPredict)):
        pred.append(np.argmax(yPredict[i]))
    #Converting one hot encoded test label to label
    test = list()
    for i in range(len(yTest)):
        test.append(np.argmax(yTest[i]))
    return my_metrics(test, pred)

def sumOfWeights(weights):
    return sum(map(sum, weights))

def getWeights(model):
    allLayersWeights=deepModel.get_weights()
    return allLayersWeights
    
# Initially train central deep model
deepModel=createDeepModel()

In [16]:
numOfIterations=2
numOfClients=2 # 10, 15, 20, 25, 30, 35, 40, 45, 50
modelLocation="Models/"+str(algoName)+"_Sync_users_"+str(numOfClients)+"_"+activationFun+"_"+optimizerName+"_FL_Model.h5"
accList, precList, recallList, f1List = [], [], [], []

deepModelAggWeights=[]
firstClientFlag=True

def updateServerModel(clientModel, clientModelWeight):
    global firstClientFlag
    for ind in range(len(clientModelWeight)):
        if(firstClientFlag==True):
            deepModelAggWeights.append(clientModelWeight[ind])            
        else:
            deepModelAggWeights[ind]=(deepModelAggWeights[ind]+clientModelWeight[ind])

def updateClientsModels():
    global clientsModelList
    global deepModel
    clientsModelList.clear()
    for clientID in range(numOfClients):
        m = keras.models.clone_model(deepModel)
        m.set_weights(deepModel.get_weights())
        clientsModelList.append(m)

In [17]:
# ----- 1. Train central model initially -----
def trainInServer():
    deepModel.fit(xServer, yServer, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # deepModel.fit(X_full, Y_full, epochs=epochs, batch_size=batch_size, verbose=verbose)
    deepModel.save(modelLocation)
trainInServer()

In [18]:
# ------- 2. Separate clients data into lists ----------
xClientsList=[]
yClientsList=[]
clientsModelList=[]
clientDataInterval=len(xClients)//numOfClients
lastLowerBound=0

for clientID in range(numOfClients):
    xClientsList.append(xClients[lastLowerBound : lastLowerBound+clientDataInterval])
    yClientsList.append(yClients[lastLowerBound : lastLowerBound+clientDataInterval])
    model=load_model(modelLocation)
    clientsModelList.append(model)
    lastLowerBound+=clientDataInterval

In [19]:
# ------- 3. Update clients' model with intial server's deep-model ----------
for clientID in range(numOfClients):
    clientsModelList[clientID].fit(xClientsList[clientID], yClientsList[clientID], epochs=epochs, batch_size=batch_size, verbose=verbose)
        
start_time = time.time()
process = psutil.Process(os.getpid())
for iterationNo in range(1,numOfIterations+1):
    print("Iteration",iterationNo)
    for clientID in range(numOfClients):
        print("clientID",clientID)
        clientsModelList[clientID].compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
        clientsModelList[clientID].fit(xClientsList[clientID], yClientsList[clientID], epochs=epochs, batch_size=batch_size, verbose=verbose)
        clientWeight=clientsModelList[clientID].get_weights()
        # Find sum of all client's model
        updateServerModel(clientsModelList[clientID], clientWeight)
        firstClientFlag=False
    #Avarage all clients model
    for ind in range(len(deepModelAggWeights)):
        deepModelAggWeights[ind]/=numOfClients

    dw_last=deepModel.get_weights()

    for ind in range(len(deepModelAggWeights)): 
        dw_last[ind]=deepModelAggWeights[ind]
     
    #Update server's model
    deepModel.set_weights(dw_last) 
    print("Server's model updated")
    print("Saving model . . .")
    deepModel.save(modelLocation)
    # Servers model is updated, now it can be used again by the clients
    updateClientsModels()
    firstClientFlag=True
    deepModelAggWeights.clear()

    yPredict = deepModel.predict(xTest)
    acc, prec, recall, f1Score= predictTestData(yPredict, yTest)
    accList.append(acc)
    precList.append(prec)
    recallList.append(recall)
    f1List.append(f1Score)
    print("Acc:\n", acc)
    print("Prec:\n", prec)
    print("Recall:\n", recall)
    print("F1-Score:\n", f1Score)

Iteration 1
clientID 0
clientID 1
Server's model updated
Saving model . . .
yPredict 10000
Accuracy  : 0.9861
Precision : 0.9861701798529342
Recall : 0.9861
f1Score : 0.9861017021601874
[[ 975    1    0    1    0    1    2    0    0    0]
 [   0 1122    2    3    1    0    3    0    4    0]
 [   2    0 1014    3    5    0    1    6    1    0]
 [   0    0    1 1008    0    0    0    0    1    0]
 [   0    0    1    1  970    0    1    0    1    8]
 [   2    0    1   10    0  874    4    0    1    0]
 [   5    2    0    1    4    1  943    0    2    0]
 [   1    0    6    4    0    0    0 1014    1    2]
 [   5    0    2    1    1    2    0    3  956    4]
 [   2    0    0    7    5    3    0    4    3  985]]
Acc:
 0.9861
Prec:
 0.9861701798529342
Recall:
 0.9861
F1-Score:
 0.9861017021601874
Iteration 2
clientID 0
clientID 1
Server's model updated
Saving model . . .
yPredict 10000
Accuracy  : 0.9865
Precision : 0.9865262126281458
Recall : 0.9865
f1Score : 0.9864958585368425
[[ 975    0 

In [20]:
memoryTraining=process.memory_percent()
timeTraining=time.time() - start_time
print("---Memory---",memoryTraining)
print("--- %s seconds (TRAINING)---" % (timeTraining))

early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')

history = deepModel.fit(xServer, yServer, epochs=epochs, 
                        validation_data = (xTest,yTest))
                        # callbacks=[early_stopping])

learningAccs=history.history['val_accuracy']
learningLoss=history.history['val_loss']

# resultSaveLocation=root_path+'Results/'+algoName+'_Users_vs_TR_vs_Iterations_vs_AccLossMemTime'+'.csv'
dfSave=pd.DataFrame(columns=['Clients', 'Iterations to converge', 'Accuracy', 'Loss', 'Memory', 'Time'])
dfSaveIndex=0
saveList = [numOfClients, len(learningLoss), learningAccs[len(learningAccs)-1], learningLoss[len(learningLoss)-1], memoryTraining, timeTraining]
dfSave.loc[dfSaveIndex] = saveList

yPredict = deepModel.predict(xTest)
acc, prec, recall, f1Score= predictTestData(yPredict, yTest)

print("Number of users:", numOfClients)
deepModel.save(modelLocation)
print("Epochs:", epochs)
print("BatchSize:", batch_size)
print("Activation:", activationFun, "Optimizer:", optimizerName)

print("Iterations:", numOfIterations)
print("Memory:", memoryTraining)
print("Time:", timeTraining)
print(dfSave)

df_performance_timeRounds = pd.DataFrame(
    {'Accuracy': accList,
     'Precision': precList,
     'Recall': recallList,
     'F1-Score': f1List 
    })

---Memory--- 10.26235883831206
--- 775.5714848041534 seconds (TRAINING)---
Epoch 1/20
375/375 [==============================] - 10s 27ms/step - loss: 0.0515 - accuracy: 0.9884 - val_loss: 0.0539 - val_accuracy: 0.9847
Epoch 2/20
375/375 [==============================] - 9s 24ms/step - loss: 0.0275 - accuracy: 0.9929 - val_loss: 0.0485 - val_accuracy: 0.9853
Epoch 3/20
375/375 [==============================] - 9s 24ms/step - loss: 0.0173 - accuracy: 0.9959 - val_loss: 0.0495 - val_accuracy: 0.9861
Epoch 4/20
375/375 [==============================] - 9s 24ms/step - loss: 0.0121 - accuracy: 0.9976 - val_loss: 0.0500 - val_accuracy: 0.9872
Epoch 5/20
375/375 [==============================] - 9s 24ms/step - loss: 0.0089 - accuracy: 0.9984 - val_loss: 0.0503 - val_accuracy: 0.9871
Epoch 6/20
375/375 [==============================] - 9s 24ms/step - loss: 0.0067 - accuracy: 0.9989 - val_loss: 0.0496 - val_accuracy: 0.9872
Epoch 7/20
375/375 [==============================] - 9s 24ms/step

In [21]:
df_performance_timeRounds

,Accuracy,Precision,Recall,F1-Score
0,0.9861,0.986170,0.9861,0.986102
1,0.9865,0.986526,0.9865,0.986496
